In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-164631
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-164631


# Optimizing  and Fine Tuning ML Pipeline




In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster



   # Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
import os

project_folder = '.'
os.makedirs(project_folder, exist_ok=True)

In [4]:
%matplotlib inline

In [5]:
%%writefile conda_dependencies.yml

dependencies:   
- python=3.6.2
- scikit-learn
- pip:
    - azureml-defaults

Writing conda_dependencies.yml


In [6]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps=RandomParameterSampling({
    "C": choice(100, 10, 1.0, 0.1, 0.01),
    "max_iter": choice(range(80,140))}
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=project_folder,
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     max_total_runs=12,
                                     max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_914f15a1-b5a6-4839-baee-fd4f672999b7
Web View: https://ml.azure.com/runs/HD_914f15a1-b5a6-4839-baee-fd4f672999b7?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-164631/workspaces/quick-starts-ws-164631&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_914f15a1-b5a6-4839-baee-fd4f672999b7
Web View: https://ml.azure.com/runs/HD_914f15a1-b5a6-4839-baee-fd4f672999b7?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-164631/workspaces/quick-starts-ws-164631&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "User errors were found in at least one of the child runs.",
    "messageFormat": null,
    "messageParameters": {},
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": null

{'runId': 'HD_914f15a1-b5a6-4839-baee-fd4f672999b7',
 'target': 'cpu-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-11-29T11:17:58.66002Z',
 'endTimeUtc': '2021-11-29T11:29:09.15223Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properti

In [10]:
assert(hyperdrive_run.get_status() == "Completed")

AssertionError: 

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

print(best_run.get_file_names())

AttributeError: 'NoneType' object has no attribute 'get_details'

In [12]:
accuracy=best_run.get_metrics()["Accuracy"]
print("Accuracy: ",accuracy)

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [ ]:
model = best_run.register_model(model_name='bankmarketing', model_path='outputs/model.joblib')

## AUTOML

pandas.core.frame.DataFrame

Uploading an estimated of 10 files
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 10
Uploading ./conda_dependencies.yml
Uploaded ./conda_dependencies.yml, 2 files out of an estimated total of 10
Uploading ./train.py
Uploaded ./train.py, 3 files out of an estimated total of 10
Uploading ./train_data.csv
Uploaded ./train_data.csv, 4 files out of an estimated total of 10
Uploading ./udacity-project.ipynb
Uploaded ./udacity-project.ipynb, 5 files out of an estimated total of 10
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-0-5-4-54-4.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-0-5-4-54-4.ipynb, 6 files out of an estimated total of 10
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-0-5-6-30-11.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-0-5-6-30-11.ipynb, 7 files out of an estimated total of 10
Uploading ./.ipynb_aml_checkpoints/udacity-project-checkpoint2021-0-5-6

$AZUREML_DATAREFERENCE_048f047ab6414807bcd7a860b0898b0a

Current provisioning state of AmlCompute is "Deleting"

